In [1]:
from __future__ import print_function, division
import os
from PIL import Image
import pandas as pd


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

In [2]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

vgg16 = models.vgg16_bn()
for param in vgg16.features.parameters():
    param.require_grad = False
class_names = ['other', 'cloud','smoke']

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

# vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/fullmodelbest.pt',map_location=torch.device('cpu')))

resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
num_features = resnet18.fc.in_features     #extract fc layers features
resnet18.fc = nn.Linear(num_features, 2) #(num_of_class == 2)

resnet34 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
num_features = resnet34.fc.in_features     #extract fc layers features
resnet34.fc = nn.Linear(num_features, 2) #(num_of_class == 2)

resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
num_features = resnet50.fc.in_features     #extract fc layers features
resnet50.fc = nn.Linear(num_features, 2) #(num_of_class == 2)


Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the

In [4]:
dir = "/home/jszumny/Arm Data/"

data = []
for file in os.listdir(dir):
    
    for image in os.listdir(dir +file):

        fullimage = Image.open(dir + "/" +  file + "/" +image)
        fullimage = fullimage.resize((1344, 1120))

        for i in range(6):
            for k in range(5):
                
                tile = fullimage.crop((i*224, k*224, (i+1)*224, (k+1)*224))
                
                vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/2features/2featuresVGG16model.pt',map_location=torch.device('cpu')))

                img_t = transform(tile)
                batch_t = torch.unsqueeze(img_t, 0)

                vgg16.eval()
                out = vgg16(batch_t)
            
                _, index = torch.max(out, 1)
            
                percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
                
                d = {"filepath": dir+file+image, "tilenum": str(i) + str(k),"class": class_names[index[0]], "percentage": percentage[index[0]].item()}
                data.append(d)

KeyboardInterrupt: 

In [7]:
df = pd.DataFrame(data)

df[df['class']=='smoke']


,filepath,tilenum,class,percentage


In [8]:
len(df[df["class"] == 'smoke'])/len(df)

0.14156735283495847

In [ ]:
df[df['class']=='smoke']


,filepath,tilenum,class,percentage
0,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,00,smoke,70.337746
5,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,10,smoke,89.789162
6,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,11,smoke,98.951363
10,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,20,smoke,65.970024
11,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,21,smoke,77.337868
...,...,...,...,...
1211,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,21,smoke,60.195953
1215,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,30,smoke,54.713535
1220,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,40,smoke,66.217506
1221,/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20...,41,smoke,80.955437


3 FEATURES
VGG16 - 20% FPR                 
Resnet18 - 3% FPR                   
Resnet34 - 14% FPR                  
Resnet50 - 0% FPR   NO TRUE POSITIVES   

UCSD - 6% FPR

In [14]:
dir = "/home/jszumny/Downloads/"


from PIL import Image
import io

fullimage = Image.open("/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20221001.001123.jpg/enatwrcamtarmacC1.a1.20221001.153223.jpg")
fullimage = fullimage.resize((2688, 2240))


def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)
vgg16 = models.vgg16_bn()

resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
num_features = resnet50.fc.in_features     #extract fc layers features
resnet50.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
num_features = resnet18.fc.in_features     #extract fc layers features
resnet18.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

resnet34 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
num_features = resnet34.fc.in_features     #extract fc layers features
resnet34.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

for param in vgg16.features.parameters():
    param.require_grad = False
class_names = ["cloud", "other", "smoke"]

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    output = vgg16.forward(tensor)
     
    probs = torch.nn.functional.softmax(output, dim=1)
    conf, classes = torch.max(probs, 1)
    print(classes.item())
    return conf.item(), class_names[classes.item()]


data = []
dir = "/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20221001.001123.jpg/"
imagetime = 0
for image in os.listdir(dir):
    if (int(image[30:36]) >= 144600) & (int(image[30:36]) <= 154600):
        imagetime = image[30:36]
        fullimage = Image.open(dir + image)
        fullimage = fullimage.resize((2688, 2240))
        

        for i in range(12):
            for k in range(10):
                vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/difference/difference.pt',map_location=torch.device('cpu')))

                vgg16.eval()

                tile = fullimage.crop((i*224, k*224, (i+1)*224, (k+1)*224))
                tile.save('/home/jszumny/Documents/test/' + str(i) + str(k) + '.jpg')
                

                image_path= "/home/jszumny/Documents/test/" + str(i) + str(k) + '.jpg'
                image = plt.imread(image_path)
                plt.imshow(image)
                
                with open(image_path, 'rb') as f:
                    image_bytes = f.read()
                    
                    conf,y_pre=get_prediction(image_bytes=image_bytes)
                    print(y_pre, ' at confidence score:{0:.2f}'.format(conf))
            
                d = {"imagetime": imagetime,"tilenum": str(i) + str(k),"class": y_pre, "percentage": '{0:.2f}'.format(conf)}
                data.append(d)

Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0


TypeError: 'list' object is not callable

In [5]:
dir = "/home/jszumny/Downloads/"


from PIL import Image
import io

fullimage = Image.open("/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20221001.001123.jpg/enatwrcamtarmacC1.a1.20221001.153223.jpg")
fullimage = fullimage.resize((2688, 2240))


def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)
vgg16 = models.vgg16_bn()

resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
num_features = resnet50.fc.in_features     #extract fc layers features
resnet50.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
num_features = resnet18.fc.in_features     #extract fc layers features
resnet18.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

resnet34 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
num_features = resnet34.fc.in_features     #extract fc layers features
resnet34.fc = nn.Linear(num_features, 3) #(num_of_class == 2)

for param in vgg16.features.parameters():
    param.require_grad = False
class_names = ["cloud", "other", "smoke"]

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    output = vgg16.forward(tensor)
     
    probs = torch.nn.functional.softmax(output, dim=1)
    conf, classes = torch.max(probs, 1)
    print(classes.item())
    return conf.item(), class_names[classes.item()]


data = []
dir = "/home/jszumny/Arm Data/enatwrcamtarmacC1.a1.20221001.001123.jpg/"
imagetime = 0
for image in os.listdir(dir):
    if (int(image[30:36]) >= 144600) & (int(image[30:36]) <= 154600):
        imagetime = image[30:36]
        fullimage = Image.open(dir + image)
        fullimage = fullimage.resize((2688, 2240))
        

        for i in range(12):
            for k in range(10):
                # vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/difference/difference.pt',map_location=torch.device('cpu')))

                # vgg16.eval()

                tile = fullimage.crop((i*224, k*224, (i+1)*224, (k+1)*224))
                tile.save('/home/jszumny/Documents/split/' +  image + "_" + str(i) + "_" + str(k) + '.jpg')
                

                # image_path= "/home/jszumny/Documents/test/" + str(i) + str(k) + '.jpg'
                # image = plt.imread(image_path)
                # plt.imshow(image)
                
                # with open(image_path, 'rb') as f:
                #     image_bytes = f.read()
                    
                #     conf,y_pre=get_prediction(image_bytes=image_bytes)
                #     print(y_pre, ' at confidence score:{0:.2f}'.format(conf))
            
                # d = {"imagetime": imagetime,"tilenum": str(i) + str(k),"class": y_pre, "percentage": '{0:.2f}'.format(conf)}
                # data.append(d)

Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /home/jszumny/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jszumny/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the

In [11]:
df = pd.DataFrame(data)
df


,imagetime,tilenum,class,percentage
0,153023,00,other,0.95
1,153023,01,smoke,0.74
2,153023,02,other,0.74
3,153023,03,smoke,0.67
4,153023,04,other,0.99
...,...,...,...,...
1033,151023,73,other,0.95
1034,151023,74,other,0.68
1035,151023,75,smoke,0.84
1036,151023,76,other,0.61


In [12]:
new = df[df['class']=='smoke']
new

,imagetime,tilenum,class,percentage
1,153023,01,smoke,0.74
3,153023,03,smoke,0.67
10,153023,10,smoke,0.98
11,153023,11,smoke,0.51
20,153023,20,smoke,0.90
...,...,...,...,...
1003,151023,43,smoke,0.56
1020,151023,60,smoke,0.76
1031,151023,71,smoke,0.88
1032,151023,72,smoke,0.51


In [13]:
smoke = df[df["tilenum"] == '34']

In [14]:
smoke

,imagetime,tilenum,class,percentage
34,153023,34,other,0.96
154,151123,34,other,0.96
274,145023,34,other,0.94
394,150723,34,other,0.96
514,152623,34,other,0.97
634,154523,34,other,0.96
754,144623,34,other,0.95
874,153123,34,other,0.97
994,151023,34,other,0.97


In [26]:
df.to_csv("ArmSmokeDetection.csv")

In [21]:
tile00  = df[df["tilenum"] == '00']
tile00

,imagetime,tilenum,class,percentage
0,153023,00,clouds,1.00
120,151123,00,clouds,1.00
240,145023,00,smoke,1.00
360,150723,00,clouds,1.00
480,152623,00,smoke,0.88
600,154523,00,clouds,1.00
720,144623,00,smoke,0.99
840,153123,00,clouds,1.00
960,151023,00,clouds,1.00
1080,145123,00,smoke,0.94


In [11]:
tile00[tile00['class']=='smoke']

,imagetime,tilenum,class,percentage
240,145023,00,smoke,0.86
1080,145123,00,smoke,0.74
1440,144723,00,smoke,0.52
1920,145223,00,smoke,0.95
2880,145323,00,smoke,0.94
3600,151923,00,smoke,0.57
3720,145423,00,smoke,0.88
4560,152223,00,smoke,0.79
4920,145523,00,smoke,0.59
5640,152323,00,smoke,0.56


In [33]:
df[df['class']=='smoke']

,imagetime,tilenum,class,percentage
0,153023,00,smoke,0.55
1,153023,01,smoke,0.69
2,153023,02,smoke,0.89
3,153023,03,smoke,0.97
10,153023,10,smoke,0.84
...,...,...,...,...
9100,150023,100,smoke,0.89
9104,150023,104,smoke,0.76
9110,150023,110,smoke,0.81
9111,150023,111,smoke,0.61


In [22]:
smoke = df[df['class'] == 'smoke']
len(smoke[(smoke.tilenum == '34') | (smoke.tilenum == '44') ]) / len(smoke)

0.0026857654431512983

In [23]:
smoke[(smoke.tilenum == '34') | (smoke.tilenum == '44') ]

,imagetime,tilenum,class,percentage
1124,145123,44,smoke,0.90
3164,144923,44,smoke,0.92
3644,151923,44,smoke,0.79


In [24]:
df.to_csv("Resnet34ARM")

In [5]:
dir = "/home/jszumny/Downloads/"


from PIL import Image
import io



def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)



def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    output = vgg16.forward(tensor)
     
    probs = torch.nn.functional.softmax(output, dim=1)
    conf, classes = torch.max(probs, 1)
    print(classes.item())
    return conf.item(), class_names[classes.item()]

def mse(img1, img2):
   h, w = img1.shape
   diff = cv2.subtract(img1, img2)
   err = np.sum(diff**2)
   mse = err/(float(h*w))
   return mse, diff



import cv2

data = []
dir = "/home/jszumny/Documents/split/"

list = []
for image in sorted(os.listdir(dir)):
    list.append(image)
list

['enatwrcamtarmacC1.a1.20221001.144623.jpg_0_0.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_1.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_2.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_3.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_4.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_5.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_6.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_7.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_8.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_0_9.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_10_0.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_10_1.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_10_2.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_10_3.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_10_4.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_10_5.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_10_6.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623.jpg_10_7.jpg',
 'enatwrcamtarmacC1.a1.20221001.144623

In [7]:
list00 = []
for i in range(len(list)):
    
    if ".jpg_0_0.jpg" in list[i]:
        
        list00.append(list[i])


        # img1 = cv2.imread(dir + list[i])
        # img2 = cv2.imread(dir + list[i-1])
        # img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        # img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

        # error, diff = mse(img1, img2)

        # cv2.imwrite("/home/jszumny/Documents/copy of smoke data/newnew/diffsmoke/" + list[i], diff)

In [15]:
list04 = []
for i in range(len(list)):
    
    if ".jpg_0_4.jpg" in list[i]:
        
        list04.append(list[i])


dir = "/home/jszumny/Documents/split/"
for i in range(len(list04)):
    
    if "144623" not in list04[i]:
        
        img1 = cv2.imread(dir + list04[i])
        img2 = cv2.imread(dir + list04[i-1])
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

        error, diff = mse(img1, img2)

        cv2.imwrite("/home/jszumny/Documents/diffsplit/" + list04[i], diff)